<a href="https://colab.research.google.com/github/Autobot37/jupyter/blob/main/JustWordsTr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-01-25 11:35:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-01-25 11:35:40 (21.1 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [ ]:
len(text)

1115394

In [ ]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:
len(chars)

65

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [ ]:
decode(encode("live fast"))

'live fast'

In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype )
print(data[:100])
print(text[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
n = int(0.9*(len(data)))
train_data = data[:n]
val_data = data[n:]






In [ ]:
batch_size = 4
x = train_data[:8]
y = train_data[1:9]
for t in range(8):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} output is {target}")


when input is tensor([18]) output is 47
when input is tensor([18, 47]) output is 56
when input is tensor([18, 47, 56]) output is 57
when input is tensor([18, 47, 56, 57]) output is 58
when input is tensor([18, 47, 56, 57, 58]) output is 1
when input is tensor([18, 47, 56, 57, 58,  1]) output is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) output is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) output is 58


In [ ]:
def get_batch():
  ix = torch.randint(len(data)-block_size,(batch_size))
  x = torch.stack([train_data[i:i+block_size] for i in ix])
  y = torchstack([train_data[i+1:i+1+block_size] for i in ix])

In [ ]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:block_size+i] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb = get_batch('train')
print("input stack")
print(xb.shape)
print(xb)

print("output stack")
print(yb.shape)
print(yb)


input stack
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
output stack
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [ ]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b][:t+1]
    target = yb[b][t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is [52] the target: 58
when input is [52, 58] the target: 1
when input is [52, 58, 1] the target: 58
when input is [52, 58, 1, 58] the target: 46
when input is [52, 58, 1, 58, 46] the target: 39
when input is [52, 58, 1, 58, 46, 39] the t

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) # (B,T,C)
        
    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_tokens):
    for _ in range(max_tokens):
        logits, loss = self(idx)
        logits = logits[:,-1,:]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx,idx_next), dim=1)
    return idx

m = BigramLLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((2,2),dtype=torch.long),max_tokens=100)[1].tolist()))



  

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


SPTWEM:v&vnnE3Q!$YqpPgiCjMzoi3uLe?O qfYWWV$NFsEsjJ-BfNlNXx
xJMLtNA R
p$peptlv-I&Q:F.O'
GvgC.:qkXenXe


In [ ]:
logits[0]

tensor([-1.5101, -0.0948,  1.0927,  0.1505,  1.6347, -0.0518,  0.4996,  0.7216,
        -0.8968, -0.4122,  1.0030,  0.8508,  0.2178,  0.0328, -0.1699,  1.0659,
        -0.6177,  1.1824,  0.0214, -0.2154, -1.4623,  2.1707,  0.1624,  1.0296,
         0.4154,  0.6207,  0.2341, -0.0326,  1.0124,  1.5122, -0.3359,  0.2456,
         1.8682,  0.7536, -0.1177, -0.1967, -0.9552, -0.8995, -0.9583, -0.5945,
         0.1321, -0.5406,  0.1405, -0.7321,  1.1796,  1.3316, -0.2094,  0.0960,
         0.9040, -0.4032,  0.3027, -0.8034, -1.2537, -1.5195,  0.7446,  1.1914,
        -0.8061, -0.6290,  1.2447, -2.4400,  0.8408, -0.3993, -0.6126, -0.6597,
         0.7624], grad_fn=<SelectBackward0>)

In [ ]:
class Bigram(nn.Module):
  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(65,100)
  def forward(self,idx,targets):
    logits = self.embedding(idx)
    B,T,C = logits.shape
    
    return logits
  def generate(self,idx,max_new):
    for _ in range(max_new):
      logits, loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_new = torch.multinomial(probs,num_samples=1)
      idx = torch.cat((idx,idx_new),dim=1)
    return idx


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)


In [ ]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad()

  loss.backward()
  optimizer.step()

print(loss.item())

4.72035551071167


In [ ]:
print(decode(m.generate(idx=torch.zeros((2,2),dtype=torch.long),max_tokens=800)[0].tolist()))




juUzL'L-aX'?OnOb?O&yZW:DWZ!BiGSrzP!LHpJpUVj;lCKtR,!oivmsuSOT'zlDxaZAJB.mgXnXE&virdV;-Kpw,a!Cfiy,usy$sda
,lviweErb:vWSPUzER?$EHLTE-UxE3:wlgXg
IvL,DnRiD3T'zo?rDqgIgCEXM&;pJRPGySP -wfUckHYqHwfBo,nprIGRFdS'QYgQYF$EQHFoiMDMgQELjKuKIoCmmIoKHxEqUZX:TDCAnMXFDJJ-wZRlg:.jFoZ;;?wOiv$NepWWT't3QyKoDWIWUFfaf'duDM?OiG:3Cj,SqEwkSmysBrVZTNcU$mhXUE
rWSb'MgBBTzkzoCttpWebHzPDFu!HAT.RUzCjelx:CJ-O'bSPufFta$Ebd;L,S.yiwNBERjsBQTbmpmIJgBU?q,siHzL,pPhuM?YVSGddl,,db?nlHSGjx&hNjKuCFBGsBcnRwFIg!W:Ph$UNs$FltKUNgjNXGJFZPynQQb?NXUxyhlyZ!cFllgC&q$ px'a!pJXE;.bXdzWdcwNfsCp$GSVXWDA$'q:3a
pSUmlapyqpJMXgmJneh$'bIMya?yevNQT:p qevUiRgX:'GJbnSPbnt&zM:pYd qOdbCOiyReYBDqeLgYwpuRLA:tRF;AUIVycLQbQ?qlLw'DIl.suDqYuZVTsE'rDhPuhDM,YKoCDWdwboQUQRT;R?OFktL'dEa!ajFZRJ!mu!zr CHhlHxP!zliOUzrgLjFCJAGf :Vzmcp PvxtWSGMU'q-Mf !Ia!W:pUiHk$JrDnkzw


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
for b in range(B):
  for r in range(T):
    x[b][t] = torch.mean(x[b][:t+1],0)

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [ ]:
wei = torch.zeros(T,T)
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
xbox3 = wei @ x
torch.allclose(xbox3,xbow2)


True

In [ ]:
B,T,C = 4,8,2
x = torch.randn(4,8,2)

head_size=16

key = nn.Linear(C, head_size,bias=False)
query = nn.Linear(C, head_size,bias=False)
value = nn.Linear(C, head_size,bias=False)
k  = key(x)
q = query(x)
v = value(x)

wei = q @ k.transpose(-2,-1) #B,T,T

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
out = wei @ v
out.shape


torch.Size([4, 8, 16])

In [ ]:
class Head(nn.Module):
  def __init__(self,head_size,n_embd=64):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size,bias=False)
    self.query = nn.Linear(n_embd, head_size,bias=False)
    self.value = nn.Linear(n_embd, head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    v = self.value(x)
    

    wei = q @ k.transpose(-2,-1)*C**-0.5 #B,T,T

    tril = torch.tril(torch.ones(T,T))
    #wei = torch.zeros((T,T))
    wei = wei.masked_fill(tril==0,float('-inf'))
    wei = F.softmax(wei,dim=-1)
    out = wei @ v
    return out

  




In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) #B,T,16
out = wei @ v # T,T @ T,16 -> T,16 
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
class MultiHead(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = [Head(head_size) for _ in range(num_heads)]

  def forward(self):
    return torch.cat([h(x) for h in self.heads],dim=1)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class Bigram(nn.Module):
  def __init__(self,n_embd=64):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.positional_encoding  = nn.Embedding(block_size, n_embd)
    self.sa_heads = MultiHead(4,n_embd//4) #still (B,T,C) 
    self.lm_head = nn.Linear(n_embd,vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb =  self.token_embedding_table(idx) # (B,T,C)
    pos_emb = self.positional_encoding(torch.arange(T)) #(T,C)
    
    x = tok_emb + pos_emb #(B,T,C)
    x = self.sa_head(x)  ##B,T,C
    logits = self.lm_head(x) #(B,T,vocab_size)

    
    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_tokens):
    for _ in range(max_tokens):
        idx_n = idx[:,-block_size]
        logits, loss = self(idx_n)
        logits = logits[:,-1,:]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx,idx_next), dim=1)
    return idx

m = Bigram(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_tokens=100)[0].tolist()))



  

In [ ]:
def scaled_dot_product(q,k,v,mask=None):
  d_k = q.size()[-1]
  attention_scores = q @ k.transpose(-2,-1)
  atten_logits = attention_scores/math.sqrt(d_k)
  if mask is not None:
    atten_logits = atten_logits.masked_fill(mask==0,float('-inf'))
  atten_logits = F.softmax(atten_logits)
  values = torch.matmul(atten_logits,v)
  return values


In [ ]:
class MultiHead(nn.Module):
  def __init__(self,input_dim,embed_dim,num_heads):
    super().__init__()
    self.inputdim = input_dim
    self.embed_dim = embed_dim
    self.head_dim =  embed_dim//num_heads

    